第二章: RAG Triad of metrics

## 一、介绍
本章节主要内容为评估RAG应用中常用的三个指标，分别为：
1. Answer relevance：评估RAG系统的输出是否与问题相关；
2. Context relevance：评估RAG系统召回的上下文是否与问题相关；
3. Groundness: 评估RAG系统的输出是否基于召回的上下文；

<img src="./images/ch03_traid.jpg" width="500">

首先需要安装本课程中需要的评估框架，如果已经安装就可以跳过这一步骤。

In [1]:
# requirements
# pip install trulens_eval

在这里，为了美观和方便展示，我们设置输出忽略警告信息。

In [2]:
# 忽略警告，避免警告影响输出结果
import warnings
warnings.filterwarnings('ignore')

接下来，导入该课程需要的工具包utils，然后设置openai的API密钥。
有三种方式设置API密钥：
1. 在环境变量中设置`OPENAI_API_KEY`，然后使用utils直接获取；
2. 显式设置api_key，直接赋值给openai.api_key；
3. 如果没有openai的密钥的话，也可以选择使用第三方服务，修改openai.api_base即可；

In [3]:
import utils
# 导入自定义的工具包

import os
import openai
# openai.api_key = utils.get_openai_api_key()
# 设置OpenAI的API密钥，从环境变量中获取

# openai.api_key = "" 
# 或者这里填入你的OpenAI API密钥

# openai.api_key = "sk- "  
# openai.api_base = " "
# 或者自定义API密钥和API基础地址，可适用第三方API服务


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


接来下，开始教程的设置环节。
首先需要重置数据库，这会在之后用于存储问题、召回结果和回答，方便管理和评估。

In [4]:
# 导入Tru类
from trulens_eval import Tru


# 实例化Tru类
tru = Tru()

# 重置数据库
# 数据库之后会用来存储问题、中间召回结果、答案以及评估结果
tru.reset_database()


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


接下来，导入读取pdf所需要的SimpleDirectoryReader，读取指定文件夹下的pdf文件。
需要注意的是，默认的参数适合读取英文文档，如果文档为中文，需要在后续将全角字符转换为半角字符。

In [5]:
# 设置Llama Index reader
from llama_index import SimpleDirectoryReader

# 从一个文件夹中读取PDF文档，然后加载到document对象中
# 使用的文档为“人工智能”词条的维基百科页面
documents = SimpleDirectoryReader(
    input_files=["./data/人工智能.pdf"]
).load_data()


为了方面起见，将读取的pdf文档加载到同一个document对象中，用`"\n\n"`隔开；

In [6]:
from llama_index import Document

# 将documents中的内容合并成一个大文档，而不是每一页都是一个文档
document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))


In [7]:
# 将中文标点符号替换成英文标点符号，方便后续处理
# 如果是英文文档，可以跳过这一步
# 不处理的话，会导致无法正确切分中文句子，会影响后续sentence_window的大小，导致输入长度大于gpt-3.5-turbo的最大限制
document.text=document.text.replace('。','. ')
document.text=document.text.replace('！','! ')
document.text=document.text.replace('？','? ')


设置index存储，首先设置用来评估的大模型为gpt-3.5-turbo，需要注意的是这里使用的版本的上下文窗口为4096，因此需要注意输入的长度。
然后设置embedding模型，我们选择了BAAI/bge-small-zh-v1.5，这里可以根据场景的需要和计算资源的trade off选择模型的大小和语种。

In [8]:
# 设置sentence_index
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

# 设置使用的大模型
# "gpt-3.5-turbo"是模型的名称
# temperature是温度，用来控制文本生成过程中的多样性
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

# 设置embedding模型
# 这里是在本地使用BAAI/bge-small-zh-v1.5
# document的所有的内容会索引到sentence index对象中
# 国内使用可以切换huggingface镜像站
sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="sentence_index"
)


使用工具包中封装好的函数，基于上一步建立好的索引，返回用于之后检索的引擎。

In [9]:
from utils import get_sentence_window_query_engine

# 根据sentence_index对象创建一个搜索引擎
# 之后会被用于在RAG应用中进行召回
sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)


这里我们先测试单个问题来debug，看一下输出是什么。

In [10]:
output = sentence_window_engine.query(
    "AI的核心问题和长远目标是什么？")
output
# 示例：使用搜索引擎进行召回

python(88138) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Response(response='AI的核心问题是建构能够跟人类类似甚至超卓的推理、知识、计划、学习、交流、感知、移动、移物、使用工具和操控机械的能力。而AI的长远目标是实现通用人工智能（GAI），即能够解决各种问题的统合强人工智能。', source_nodes=[NodeWithScore(node=TextNode(id_='a30ea2a3-2e08-4ebc-a9f3-4627ebb6b6e0', embedding=None, metadata={'window': '⼈⼯智能的研究是⾼度技术性和专业的，各分⽀领域都是深⼊且各不相通的，因⽽涉及范围极⼴[9].  ⼈⼯智能的\n研究可以分为⼏个技术问题.  其分⽀领域主要集中在解决具体问题，其中之⼀是，如何使⽤各种不同的⼯具完成\n特定的应⽤程序. \n AI的核⼼问题包括建构能够跟⼈类似甚⾄超卓的推理、知识、计划、学习、交流、感知、移动 、移物、使⽤⼯\n具和操控机械的能⼒等[10].  通⽤⼈⼯智能（GAI）⽬前仍然是该领域的长远⽬标[11].  ⽬前弱⼈⼯智能已经有初\n步成果，甚⾄在⼀些影像识别、语⾔分析、棋类游戏等等单⽅⾯的能⼒达到了超越⼈类的⽔平，⽽且⼈⼯智能的\n通⽤性代表着，能解决上述的问题的是⼀样的AI程序，⽆须重新开发算法就可以直接使⽤现有的AI完成任务，与\n⼈类的处理能⼒相同，但达到具备思考能⼒的统合强⼈⼯智能还需要时间研究，⽐较流⾏的⽅法包括统计⽅法，\n计算智能和传统意义的AI. ', 'original_text': 'AI的核⼼问题包括建构能够跟⼈类似甚⾄超卓的推理、知识、计划、学习、交流、感知、移动 、移物、使⽤⼯\n具和操控机械的能⼒等[10]. '}, excluded_embed_metadata_keys=['window', 'original_text'], excluded_llm_metadata_keys=['window', 'original_text'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bf662866-5fd5-43b5-8e76-a021937ebb56', node_type=<ObjectType.DOCUMENT: '4'

In [11]:
# 在实际开发中，可以通过查看metadata进行debug
output.metadata

{'a30ea2a3-2e08-4ebc-a9f3-4627ebb6b6e0': {'window': '⼈⼯智能的研究是⾼度技术性和专业的，各分⽀领域都是深⼊且各不相通的，因⽽涉及范围极⼴[9].  ⼈⼯智能的\n研究可以分为⼏个技术问题.  其分⽀领域主要集中在解决具体问题，其中之⼀是，如何使⽤各种不同的⼯具完成\n特定的应⽤程序. \n AI的核⼼问题包括建构能够跟⼈类似甚⾄超卓的推理、知识、计划、学习、交流、感知、移动 、移物、使⽤⼯\n具和操控机械的能⼒等[10].  通⽤⼈⼯智能（GAI）⽬前仍然是该领域的长远⽬标[11].  ⽬前弱⼈⼯智能已经有初\n步成果，甚⾄在⼀些影像识别、语⾔分析、棋类游戏等等单⽅⾯的能⼒达到了超越⼈类的⽔平，⽽且⼈⼯智能的\n通⽤性代表着，能解决上述的问题的是⼀样的AI程序，⽆须重新开发算法就可以直接使⽤现有的AI完成任务，与\n⼈类的处理能⼒相同，但达到具备思考能⼒的统合强⼈⼯智能还需要时间研究，⽐较流⾏的⽅法包括统计⽅法，\n计算智能和传统意义的AI. ',
  'original_text': 'AI的核⼼问题包括建构能够跟⼈类似甚⾄超卓的推理、知识、计划、学习、交流、感知、移动 、移物、使⽤⼯\n具和操控机械的能⼒等[10]. '},
 'c5720668-54f2-46dc-b7d9-499ddc7bf437': {'window': '⼈⼯智能的\n研究可以分为⼏个技术问题.  其分⽀领域主要集中在解决具体问题，其中之⼀是，如何使⽤各种不同的⼯具完成\n特定的应⽤程序. \n AI的核⼼问题包括建构能够跟⼈类似甚⾄超卓的推理、知识、计划、学习、交流、感知、移动 、移物、使⽤⼯\n具和操控机械的能⼒等[10].  通⽤⼈⼯智能（GAI）⽬前仍然是该领域的长远⽬标[11].  ⽬前弱⼈⼯智能已经有初\n步成果，甚⾄在⼀些影像识别、语⾔分析、棋类游戏等等单⽅⾯的能⼒达到了超越⼈类的⽔平，⽽且⼈⼯智能的\n通⽤性代表着，能解决上述的问题的是⼀样的AI程序，⽆须重新开发算法就可以直接使⽤现有的AI完成任务，与\n⼈类的处理能⼒相同，但达到具备思考能⼒的统合强⼈⼯智能还需要时间研究，⽐较流⾏的⽅法包括统计⽅法，\n计算智能和传统意义的AI.  ⽬前有⼤量的⼯具应⽤了⼈⼯智能，其中包括搜索和数学优化、逻辑推演. ',
  'or

## 二、 Feedback functions
feedback function是一个衡量RAG系统的问题、上下文、答案三者之间关系的函数。在RAG系统中，feedback function通常是一个评估模型的指标，用于评估RAG系统的各个方面的性能。具体来说，在本教程中，主要为，answer relevance、context relevance、groundness三个指标。

<img src="./images/ch03_feedback.jpg" width="500">


首先设置基础配置，保证之后可以可视化，方便分析结果。

In [12]:
import nest_asyncio

# 保证后续可以使用streamlit进行评估结果管理和可视化
nest_asyncio.apply()


In [13]:
from trulens_eval import OpenAI as fOpenAI

# 初始化OpenAI gpt-3.5-turbo模型作为provider
# provider之后会用来辅助评估RAG应用的各个指标：answer relevance, context relevance, groundedness.
provider = fOpenAI()

### 2.1、 Answer Relevance
answer relevance用来评估RAG系统的输出是否与问题相关。

<img src="./images/ch03_answer_rele.jpg" width="500">


answer relevance的feedback function的结构为：

<img src="./images/ch03_structure_answer.jpg" width="500">

这里使用封装好的Feedback函数即可，我们需要做的是：指定评估的方式，指定名称，以及评估的对象。

In [14]:
from trulens_eval import Feedback


# 这里为answer relevance设置feedback
# 使用provider.relevance_with_cot_reasons作为评估函数，即，通过调用LLM使用chain of thought的方式进行评估
# on_input_output()表示在输入和输出上进行评估
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2.2、 Context Relevance
context relevance用来评估RAG系统召回的上下文是否与问题相关。

<img src="./images/ch03_context_rele.jpg" width=500>

其feedback function的结构为：

<img src="./images/ch03_structure_context.jpg" width=500>

In [15]:
from trulens_eval import TruLlama

# 选择召回的上下文
context_selection = TruLlama.select_source_nodes().node.text

这里的设置和上一步类似，只需要修改评估的对象即可。
也可以选择修改评估的方式，进行对比。

In [16]:
import numpy as np


# 使用provider.qs_relevance作为评估函数
# on_input()表示在输入上进行评估
# on(context_selection)表示在context_selection上进行评估
# aggregate(np.mean)表示使用np.mean作为聚合函数
# 这里实际的意思是：对于context_selection中的每个句子，都会进行评估，然后取平均值作为最终的评估结果
f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [17]:
import numpy as np

# 同上，对于context_selection中的每个句子进行评估，取平均值作为评估结果
f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)


✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 2.3、 Groundedness

In [18]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

最后是groundedness，用来评估RAG系统的输出是否基于召回的上下文。
设置和之前的类似。

In [19]:
# groundedness的评估，解释同answer relevance和context relevance
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## 三、Evaluation of the RAG application

在RAG系统的评估中，feedback function可以通过多种方式实现。
使用人工打分的方法可以获得最准确的评估结果，但是这种方法成本较高，因此在实际应用中，通常使用自动评估的方法。
在本教程中，使用gpt-3.5-turbo来对RAG系统进行评估。
这种方法的好处是，可以快速、低成本地对RAG系统进行评估，但是其评估结果可能不如人工打分准确。

<img src="./images/ch03_bench.jpg" width="500">

下面是整个RAG系统的评估流程的实现。

In [20]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode


# 实例化TruLlama类，用来记录评估结果
# sentence_window_engine是之前创建的搜索引擎
# app_id是应用的ID，用来标识应用
tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

读取用来评估的问题，这里为了节约时间并降低调用API的成本，我们只设置了6个问题。
在实际场景中，可以手写或通过prompt seed的方法生成更多的问题，覆盖更多的场景。

In [21]:
eval_questions = []
# 读取评估问题，在./data/eval_questions.txt下，可以自定义
with open('./data/eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [22]:
eval_questions

['人工智能中的先验知识是如何被存储的？',
 '人工智能的自我更新和自我提升是否可能导致其脱离人类的控制？',
 '管理者如何管理AI？',
 '强人工智能是什么？',
 '人工智能被滥用带来的危害？']

In [23]:
eval_questions.append("如何在人工智能领域获得成功？")

In [24]:
eval_questions

['人工智能中的先验知识是如何被存储的？',
 '人工智能的自我更新和自我提升是否可能导致其脱离人类的控制？',
 '管理者如何管理AI？',
 '强人工智能是什么？',
 '人工智能被滥用带来的危害？',
 '如何在人工智能领域获得成功？']

接下来开始评估，请求RAG系统的输出，然后使用feedback function对输出进行评估。

In [25]:
# 对于每个评估问题，进行评估，并记录结果
# 注意：该过程可能会比较耗时，请耐心等待
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

之后，需要进行编解码，将评估结果转换为中文可读的形式，方便分析。

In [26]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

# 将记录中的unicode转换成中文，方便查看
def decode_unicode(s):
    return s.encode('ascii').decode('unicode-escape')

records['input'] = records['input'].apply(decode_unicode)
records['output'] = records['output'].apply(decode_unicode)

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_47c48da5823443781893d80f6f67a7a4,"""人工智能中的先验知识是如何被存储的？""","""先验知识是通过某种方式告诉给机器的知识。它可以包括描述目标、特征、种类及对象之间关系的知识...",-,"{""record_id"": ""record_hash_47c48da582344378189...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-02T16:25:03.198488"", ""...",2024-02-02T16:25:06.126069,0.7,0.85,1.0,"[{'args': {'prompt': '人工智能中的先验知识是如何被存储的？', 're...","[{'args': {'question': '人工智能中的先验知识是如何被存储的？', '...",[{'args': {'source': '知识表⽰是⼈⼯智能领域的核⼼研究问题之⼀，它的⽬...,2,0,0.0
1,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1cb697249d410e873b043ec5c2359667,"""人工智能的自我更新和自我提升是否可能导致其脱离人类的控制？""","""人工智能的自我更新和自我提升可能导致其脱离人类的控制。""",-,"{""record_id"": ""record_hash_1cb697249d410e873b0...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-02T16:25:06.255939"", ""...",2024-02-02T16:25:08.118188,1.0,0.50,1.0,[{'args': {'prompt': '人工智能的自我更新和自我提升是否可能导致其脱离人...,[{'args': {'question': '人工智能的自我更新和自我提升是否可能导致其脱...,[{'args': {'source': '⾄少，它本⾝应该有正常的情绪. ⼀个⼈⼯智能...,1,0,0.0
2,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_a621afecc98501ca393d7a9436e9c7e0,"""管理者如何管理AI？""","""管理者可以通过以下几点来管理AI：\n1. 放弃行政工作：将一些繁琐的行政工作交给AI来处...",-,"{""record_id"": ""record_hash_a621afecc98501ca393...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-02T16:25:08.227341"", ""...",2024-02-02T16:25:15.509122,0.9,0.55,NaN,"[{'args': {'prompt': '管理者如何管理AI？', 'response':...","[{'args': {'question': '管理者如何管理AI？', 'statemen...",NaN,7,0,0.0
3,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_20cb24147006b1390f35b70f6488f1ff,"""强人工智能是什么？""","""强人工智能是一种观点，认为计算机不仅仅是用来研究人的思维的工具，而是具有自主思维能力的存在...",-,"{""record_id"": ""record_hash_20cb24147006b1390f3...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-02T16:25:15.603986"", ""...",2024-02-02T16:25:18.277671,0.9,0.75,NaN,"[{'args': {'prompt': '强人工智能是什么？', 'response': ...","[{'args': {'question': '强人工智能是什么？', 'statement...",NaN,2,0,0.0
4,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_242b3188def87c7dccea8a44deb172ad,"""人工智能被滥用带来的危害？""","""人工智能被滥用可能带来的危害包括违反法律，如著作权法，以及涉及色情影片的处理等相关案件。此...",-,"{""record_id"": ""record_hash_242b3188def87c7dcce...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-02T16:25:18.371248"", ""...",2024-02-02T16:25:23.671531,0.8,NaN,NaN,"[{'args': {'prompt': '人工智能被滥用带来的危害？', 'respons...",NaN,NaN,5,0,0.0


In [27]:
import pandas as pd

# 展示评估结果
pd.set_option("display.max_colwidth", None)
display(records[["input", "output"] + feedback])

,input,output,Answer Relevance,Groundedness,Context Relevance
0,"""人工智能中的先验知识是如何被存储的？""","""先验知识是通过某种方式告诉给机器的知识。它可以包括描述目标、特征、种类及对象之间关系的知识，也可以描述事件、时间、状态、原因和结果，以及任何需要机器存储的知识。""",0.7,1.0,0.85
1,"""人工智能的自我更新和自我提升是否可能导致其脱离人类的控制？""","""人工智能的自我更新和自我提升可能导致其脱离人类的控制。""",1.0,1.0,0.50
2,"""管理者如何管理AI？""","""管理者可以通过以下几点来管理AI：\n1. 放弃行政工作：将一些繁琐的行政工作交给AI来处理，以便管理者能够更专注于更重要的任务。\n2. 退守分析预测的领域而强化自己的综合判断力：AI在分析和预测方面具有很高的能力，管理者可以将一些数据分析和预测的工作交给AI，而自己则需要加强自己的综合判断力，以便更好地利用AI的结果做出决策。\n3. 把AI当作同事，形成协同合作的团队：管理者可以将AI视为团队的一员，与AI进行协同合作，共同解决问题和实现目标。\n4. 了解AI的局限性：管理者需要明白AI虽然在某些方面具有很高的能力，但仍然存在一些难以克服的瓶颈，例如无法具备创意、智能、同情心和审美等能力。因此，管理者需要在使用AI时注意其局限性，并做出相应的调整和决策。""",0.9,NaN,0.55
3,"""强人工智能是什么？""","""强人工智能是一种观点，认为计算机不仅仅是用来研究人的思维的工具，而是具有自主思维能力的存在。根据这个观点，只要运行适当的程序，计算机本身就能具备思维能力。""",0.9,NaN,0.75
4,"""人工智能被滥用带来的危害？""","""人工智能被滥用可能带来的危害包括违反法律，如著作权法，以及涉及色情影片的处理等相关案件。此外，人工智能的发展也可能导致一些职业被机器和人工智能取代，直接影响就业市场。还有一些专家担心，人工智能的发展可能会导致无法控制的局面，甚至可能研发出人们无法理解的武器。因此，人工智能的滥用可能会带来社会和安全方面的危害。""",0.8,NaN,NaN
5,"""如何在人工智能领域获得成功？""","""在人工智能领域获得成功的方法是通过模仿人类的推理方式，并利用概率和经济学的概念来处理不确定或不完整的信息。此外，寻找更有效的算法也是人工智能研究的重点。人类解决问题的模式通常是使用最快捷、直观的判断，而不是有意识的、一步一步的推导。另外，研究者还可以探索将人类智能简化成符号处理的方法。""",NaN,NaN,NaN


In [28]:
# 获取leaderboard
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
App_1,0.86,1.0,0.6625,3.333333,0.0


In [29]:
# 运行dashboard
# 注意：请检查端口是否被占用，如果被占用，请修改端口号
tru.run_dashboard()

Starting dashboard ...


python(88247) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.2.7:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>